In [36]:
# Dependencias

import tkinter as tk
from tkinter import filedialog
import os
import numpy as np
import gmsh
import sys
from math import pi

In [37]:
# Funciones

# Function to check if a file exists and delete it
def check_and_delete_file(file_path):
    # Check if the given path is just a file name
    if not os.path.isabs(file_path):
        # Prepend the current working directory to the file name
        file_path = os.path.join(os.getcwd(), file_path)

    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted existing file: {file_path}")

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    file_selected = filedialog.askopenfilename()
    if not file_selected:
        print("No folder selected. Exiting...")
        sys.exit(1)  # Exit the script with a non-zero exit code
    return file_selected

def getSizeLimits(elementTuple, printLog=False):
    """
    Given an element dimension and element tag, it returns the xy limits.

    Args:
        elementTuple (tuple): A tuple containing (elementDim, elementTag).
        printLog (bool, optional): If True, prints the limits. Defaults to False.

    Returns:
        list: A list containing the (x,y) pairs for min and max values.
    """
    elementDim = elementTuple[0]
    elementTag = elementTuple[1]
    xmin, ymin, _, xmax, ymax, _ = gmsh.model.occ.getBoundingBox(elementDim, elementTag)
    boundingBoxDimensions = [(xmin, ymin), (xmax, ymax)]
    if printLog:
        print(f'The minimum dimension coordinates of element {elementTag} are {boundingBoxDimensions[0]} and the maximum dimensions are {boundingBoxDimensions[1]}')
    return boundingBoxDimensions

def getSizeLimitsMultipleEntities(entitiesList, printLog=False):
    """
    Calculates the overall bounding box for multiple entities.

    Args:
        entitiesList (list): A list of tuples, each containing (elementDim, elementTag) for an entity.
        printLog (bool, optional): If True, prints individual entity limits. Defaults to False.

    Returns:
        list: A list containing two tuples: [(min_x, min_y), (max_x, max_y)] representing the overall bounding box for all entities.
    """
    limits_array = []
    for entity in entitiesList:
        limits = getSizeLimits(entity, printLog=printLog)
        limits_array.append(limits)
        
    # Extract all x and y values using a list comprehension
    all_points = [(x, y) for sublist in limits_array for x, y in sublist]

    # Unzip the list of tuples into separate lists
    all_x_values, all_y_values = zip(*all_points)

    # Find min and max for x and y values
    min_x, max_x = min(all_x_values), max(all_x_values)
    min_y, max_y = min(all_y_values), max(all_y_values)

    if printLog:
        print(f'For all {len(entitiesList)} entities, the min x is {min_x}, the max x is {max_x}, the min y is {min_y}, the max y is {max_y}')
    
    boundingCoordinates = [(min_x, min_y), (max_x, max_y)]
    
    return boundingCoordinates

def getLineIntersection(p1, p2, surfaceTuple, printLog=True):
    """
    Creates a line between two points and finds intersections with a given surface.

    Args:
        p1 (list): First point coordinates [x, y].
        p2 (list): Second point coordinates [x, y].
        surfaceTuple (tuple): Gmsh tuple (dim, tag) for the tool element.
        printLog (bool, optional): If True, prints intersection info. Defaults to True.

    Returns:
        list: A list containing the tuples (dim, tag) for the intersecting elements.
    """
    x0, y0 = p1[0], p1[1]
    x1, y1 = p2[0], p2[1]
    p_start = gmsh.model.occ.addPoint(x0, y0, 0)
    p_end = gmsh.model.occ.addPoint(x1, y1, 0)
    line = gmsh.model.occ.addLine(p_start, p_end)
    lineInfo, _ = gmsh.model.occ.intersect([(1, line)], surfaceTuple, removeObject=True, removeTool=False)
    
    if printLog:
        print(f'Intersection line tuples (dim, tag): {lineInfo}')

    return lineInfo

def getLineEndpoints(lineTuple, printLog=False):
    """
    Gets the coordinates of the beginning and end points of a line.

    Args:
        lineTuple (tuple): A tuple containing (dim, tag) of the line.
        printLog (bool, optional): If True, prints the coordinates. Defaults to False.

    Returns:
        tuple: Coordinates of the start and end points (start_coords, end_coords).
    """
    lineTag = lineTuple[1]
    boundary = gmsh.model.getBoundary([lineTuple], combined=False, oriented=False, recursive=False)
    start_point_tag = boundary[0][1]
    end_point_tag = boundary[1][1]
    x_start, y_start, _, _, _, _ = gmsh.model.getBoundingBox(0, start_point_tag)
    x_end, y_end, _, _, _, _ = gmsh.model.getBoundingBox(0, end_point_tag)
    
    start_point_coords = [x_start, y_start]
    end_point_coords = [x_end, y_end]
    
    if printLog:
        print(f'The line tag: {lineTag} starts at p1-{start_point_tag}: {start_point_coords} and ends at p2-{end_point_tag}: {end_point_coords}')
    
    return start_point_coords, end_point_coords

def getLineEndpointsMultipleEntities(entitiesList, printLog=False):
    """
    Gets the coordinates of the beginning and end points for multiple lines.

    Args:
        entitiesList (list): A list of tuples, each containing (dim, tag) for a line.
        printLog (bool, optional): If True, prints the coordinates array. Defaults to False.

    Returns:
        list: A list containing coordinates for the start and end points of each line.
    """
    coordinatesArray = []
    for entity in entitiesList:
        startPoint, endPoint = getLineEndpoints(entity, printLog=printLog)
        coordinatesArray.append([startPoint, endPoint])
    
    if printLog:
        print(f'The line start & end coordinates array is: {coordinatesArray}')
    
    return coordinatesArray

def createCoordinatesList(pointArray, surfaceEntities, printLog=False):
    """
    Creates a list of coordinates for multiple line intersections.

    Args:
        pointArray (list): A list of point pairs defining lines.
        printLog (bool, optional): If True, prints the coordinates list. Defaults to False.

    Returns:
        list: A list containing the coordinates of intersecting lines.
    """
    entities=surfaceEntities
    coordinatesList = []
    for points in pointArray:
        lineEntities = getLineIntersection(points[0], points[1], entities, printLog=False)
        gmsh.model.occ.synchronize()
        coordinates = getLineEndpointsMultipleEntities(lineEntities, printLog=False)
        coordinatesList.append(coordinates)
        
    if printLog:
        print(f'The line coordinate list for the point array is: {coordinatesList}')
        
    return coordinatesList

def createPointArray(numberLines, sizeLimits):
    """
    Creates an array of points defining lines across the bounding box limits.

    Args:
        numberLines (int): Number of lines to create.
        sizeLimits (list): A list containing two tuples [(min_x, min_y), (max_x, max_y)] representing the bounding box limits.

    Returns:
        list: A list of point pairs defining lines.
    """
    ymin = sizeLimits[0][1]
    ymax = sizeLimits[1][1]
    xmin = sizeLimits[0][0]
    xmax = sizeLimits[1][0]
    
    delta=(ymax-ymin)/(numberLines)
    print(f'El espaciamiento de la capa es de {np.round(delta,2)}')
    
    yCoordinates = np.linspace(ymin+delta/2, ymax-delta/2, numberLines)
    pointArray = [
        [[xmin, y], [xmax, y]]
        for y in yCoordinates
    ]
    
    return pointArray

def createZpoints(numberLayers, layerHeigth):
    """_summary_

    Args:
        numberLayers (int): number of layers
        layerHeigth (int): layer heigth

    Returns:
        ndarray: layer z coordinate array
    """
    zpointArray=np.linspace(layerHeigth, layerHeigth*numberLayers, numberLayers)
    return zpointArray

In [38]:
# Clases

class GCodeGenerator:
    def __init__(self, name, flowRate=600, extrusionHeight=5, nozzleDiameter=9, default_position_delta=[0,0,0], baseCoordinates=[70,35]):
        """
        Initialize the GCodeGenerator with default feed rate and extrusion rate.
        
        Args:
            feed_rate (int): The default speed of the printer head in mm/min.
            extrusion_rate (float): The default rate of extrusion.
        """
        self.gcode_lines = []
        self.name=name
        self.flowRate = flowRate
        self.extrusionHeight = extrusionHeight
        self.nozzleDiameter = nozzleDiameter
        self.default_position_delta=default_position_delta
        self.baseCoordinates=baseCoordinates
    
    def calculate_extrusion_factor(self, f, L):
        # SE NECESITA VERIFICAR ESTA FORMULA
        h=self.extrusionHeight
        F=self.flowRate
        phi=self.nozzleDiameter
        E=(4*h*f*L*phi)/(pi*1.75**2)
        return E
    
    def clear_all(self):
        """Clear all the G-code commands."""
        self.gcode_lines = []
        
    def default_commands(self):
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; Set all axes to home')
        self.gcode_lines.append('G28') # Set all axes to home
        self.gcode_lines.append('; Establish units commands')
        self.gcode_lines.append('G21') # Establish units commands
        self.gcode_lines.append('; Establish absolute extrusion coordinates')
        self.gcode_lines.append('M82') # Establish units commands
        self.gcode_lines.append('; ===========================================')
        
    def close_commands(self):
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; End all')
        self.gcode_lines.append('M2') # End all
        self.gcode_lines.append('; ===========================================')
        
    def set_absolute_coordinates(self):
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; Set absolute coordinates')
        self.gcode_lines.append('G90') # Set absolute coordinates
        self.gcode_lines.append('; ===========================================')
        
    def set_relative_coordinates(self):
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; Set relative coordinates')
        self.gcode_lines.append('G91') # Set relative coordinates
        self.gcode_lines.append('; ===========================================')
    
    def set_coordinates_as_base(self):
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; Set current coordinates as base coordinates')
        self.gcode_lines.append('G92') # Set current coordinates as base coordinates
        self.gcode_lines.append('; ===========================================')
        
    def set_coordinates_to_printer_limits(self, coordinates_list):
        new_coordList = []
        base_coords=self.baseCoordinates
        for row in coordinates_list:
            new_row = []
            for col in row:
                new_col = []
                for coord in col:
                    new_coord = [coord[0] + base_coords[0], coord[1] + base_coords[1]]
                    new_col.append(new_coord)
                new_row.append(new_col)
            new_coordList.append(new_row)
            
        return new_coordList
    
    def go_home(self, coordinates_list):
        
        xo, yo = self.baseCoordinates
        self.coord_list=self.set_coordinates_to_printer_limits(coordinates_list)
        
        self.gcode_lines.append('; ===========================================')
        self.gcode_lines.append('; Set current coordinates to ')
        pass_string=f'G0 X{xo} Y{yo} F{self.flowRate}'
        self.gcode_lines.append(pass_string)
        self.gcode_lines.append('; ===========================================')
    
    def generate_gcode_from_coordinates_for_layer(self, z_coord, reverse=False, f=1.10):
        
        if reverse is True:
            coord_list = list(reversed(self.coord_list))
        else:
            coord_list=self.coord_list
        
        # Se debe adicionar lineas de G0 en los huecos en vez de G1
        
        for index, row in enumerate(coord_list):
            #print(index)
            pass_string=f'; Fila numero {index}'
            self.gcode_lines.append(pass_string)
            
            if index == 0:
                base_coord=self.baseCoordinates
                vector_last=np.array(base_coord)
            
            if index % 2 == 0:
                for line in row:
                    for coord in line:
                        xo=coord[0]
                        yo=coord[1]
                        vector_new=np.array([xo,yo])
                        vector=vector_new-vector_last
                        L = np.linalg.norm(vector)
                        e=self.calculate_extrusion_factor(f,L)
                        pass_string=f'G1 X{xo} Y{yo} Z{z_coord} E{e} F{self.flowRate}'
                        self.gcode_lines.append(pass_string)
                        vector_last=vector_new
            else:
                for line in reversed(row):
                    for coord in reversed(line):
                        xo=coord[0]
                        yo=coord[1]
                        vector_new=np.array([xo,yo])
                        vector=vector_new-vector_last
                        L = np.linalg.norm(vector)
                        e=self.calculate_extrusion_factor(f,L)
                        pass_string=f'G1 X{xo} Y{yo} Z{z_coord} E{e} F{self.flowRate}'
                        self.gcode_lines.append(pass_string)
                        vector_last=vector_new

    
    def save_gcode_to_file(self):
        """
        Save the generated G-code to a file.
        
        Args:
            file_path (str): Path to the file where G-code will be saved.
        """
        file_path=self.name
        check_and_delete_file(file_path)
        with open(file_path, 'w') as f:
            f.write('\n'.join(self.gcode_lines))
            f.write('\n')  

In [39]:
file_brep=select_file()
print(file_brep)

C:/Users/nmora/Desktop/gcode_prueba/Prueba.geo


In [40]:
#=====================================
# CREAMOS ARCHIVO G CODE
#=====================================

"""
Dado un archivo con geometria: .brep o .geo

Returns:
    _type_: _description_
"""

# Initialize Gmsh
gmsh.initialize()

gmsh.open(file_brep)

# Synchronize to update the internal Gmsh data structures
gmsh.model.occ.synchronize()

# get the surfaces entities in the model
entities = gmsh.model.occ.getEntities(dim=2)
print(f'There are {len(entities)} entities in the model: {entities}')

# get the bounding box size limits
sizeLimits=getSizeLimitsMultipleEntities(entities, printLog=False)
print(sizeLimits)

# Create point array
pointArray=createPointArray(numberLines=22, sizeLimits=sizeLimits)

# create the line coordinates begining and end coordinates per vertical coordinate
coordList=createCoordinatesList(pointArray, entities, printLog=False)

# Visualize the model
model_visibility = True
if model_visibility:
    gmsh.fltk.run()

# Finalize Gmsh
gmsh.finalize()

There are 1 entities in the model: [(2, 1)]
[(-1e-07, -1e-07), (250.0000001, 190.0000001)]
El espaciamiento de la capa es de 8.64


In [41]:
# Datos
numero_capas_verticales=8
espesor_capa_verticales=5
nombre_archivo='prueba'
coordenadas_origen=[50,50]

In [42]:

zPoints=createZpoints(numero_capas_verticales,espesor_capa_verticales)

gcode_gen = GCodeGenerator(nombre_archivo, baseCoordinates=coordenadas_origen)
gcode_gen.clear_all()
gcode_gen.default_commands()
gcode_gen.set_absolute_coordinates()

gcode_gen.go_home(coordList)

for i, zCoord in enumerate(zPoints):
    if i % 2 == 0:
        gcode_gen.generate_gcode_from_coordinates_for_layer(zCoord)
    else:
        gcode_gen.generate_gcode_from_coordinates_for_layer(zCoord, reverse=True)
    
gcode_gen.close_commands()
gcode_gen.save_gcode_to_file()

[54.31818172272727, 54.31818172272727]
76.81750927572728
1580.8819531842773
[253.5984847689394, 54.318181722727275]
199.2803030462121
4101.130558399869
[71.59090901363636, 71.59090901363636]
12.21366259698776
251.3536166937503
[267.9924241780303, 71.59090901363636]
196.40151516439397
4041.885942786564
[88.86363630454545, 88.86363630454545]
12.213662596987751
251.3536166937501
[282.3863635871212, 88.86363630454545]
193.52272728257574
3982.641327173258
[86.7727272809091, 106.13636359545454]
8.807397350931188
181.25367065252723
[296.78030299621213, 106.13636359545455]
210.00757571530303
4321.894703571278
[83.31818182272727, 123.40909088636364]
8.807397350931216
181.2536706525278
[170.0, 123.40909088636364]
86.68181817727273
1783.8865555219334
[79.86363636454546, 140.68181817727273]
8.807397350931199
181.2536706525274
[170.0, 140.68181817727273]
90.13636363545454
1854.9800942578995
[76.40909090636363, 157.95454546818183]
8.807397350931202
181.25367065252752
[170.0, 157.95454546818183]
93.5